## Movies Recommendation System 

In [1]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime

## Local load proof

itm_file = pd.read_csv('movies.dat', sep='::', engine='python')
use_file = pd.read_csv('users.dat', sep='::', engine='python')
rat_file = pd.read_csv('ratings.dat', sep='::', engine='python')

itm_file.columns = ['movie_id', 'movie_title', 'genre']
use_file.columns = ['userid', 'twitter_id']
rat_file.columns = ['user_id','movie_id','rating','rating_timestamp']

## Load proofs from an URL

url="https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat"
c=pd.read_csv(url, sep='::',encoding='utf8'm,error_bad_lines=False,engine='python')
c.columns = ['movie_id', 'movie_title', 'genre']
c

## Beginning  
* Save the URLs in some temporary variable.  
* Convert the dataset that is saved in the temporal variables to Data Frames.  
* Add appropriate names to the columns.

In [2]:
url_items = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat"
url_ratin = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat"
url_users = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/users.dat"

itm_file = pd.read_csv(url_items, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
use_file = pd.read_csv(url_users, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
rat_file = pd.read_csv(url_ratin, sep='::',encoding='utf8',error_bad_lines=False,engine='python')

itm_file.columns = ['movie_id', 'movie_title', 'genre']
use_file.columns = ['userid', 'twitter_id']
rat_file.columns = ['user_id','movie_id','rating','rating_timestamp']

## Items dataset

In [3]:
itm_file.head() 

,movie_id,movie_title,genre
0,10,La sortie des usines Lumière (1895),Documentary|Short
1,12,The Arrival of a Train (1896),Documentary|Short
2,25,The Oxford and Cambridge University Boat Race ...,NaN
3,91,Le manoir du diable (1896),Short|Horror
4,131,Une nuit terrible (1896),Short|Comedy|Horror


In [4]:
itm_file.shape

(34513, 3)

It'd be noticed that the column *movie_title* contains other important information furthermore the title. Then it is necessary to put apart the realisation year of the movie in another column. Finally, of course, it is very important to errase characters <<**(**>> and << **)**>>

In [5]:
itm_file['movie_year'] = itm_file['movie_title']

In [6]:
itm_file['movie_title'] = itm_file['movie_title'].str.split('(').str[0]
itm_file['movie_title'] = itm_file['movie_title'].str.strip()
itm_file['movie_year'] = itm_file['movie_year'].str[-6:]

In [7]:
itm_file['movie_year'] = itm_file['movie_year'].str.split('(').str[1]
itm_file['movie_year'] = itm_file['movie_year'].str.split(')').str[0]

In [8]:
itm_file.head() 

,movie_id,movie_title,genre,movie_year
0,10,La sortie des usines Lumière,Documentary|Short,1895
1,12,The Arrival of a Train,Documentary|Short,1896
2,25,The Oxford and Cambridge University Boat Race,NaN,1895
3,91,Le manoir du diable,Short|Horror,1896
4,131,Une nuit terrible,Short|Comedy|Horror,1896


The dataset is almost ready but before at all, it's a good practice to check if there are null values. In this case have null values could mean conflicts in our future transformations.

In [9]:
itm_file.isnull().values.any()

True

In [10]:
itm_file.isnull().sum()

movie_id         0
movie_title      0
genre          279
movie_year       0
dtype: int64

Effectively theres are null values, however it is easy to fill the null values with some other specific value. 

In [11]:
itm_file = itm_file.fillna(value = 'noGenre')

In [12]:
itm_file.head() 

,movie_id,movie_title,genre,movie_year
0,10,La sortie des usines Lumière,Documentary|Short,1895
1,12,The Arrival of a Train,Documentary|Short,1896
2,25,The Oxford and Cambridge University Boat Race,noGenre,1895
3,91,Le manoir du diable,Short|Horror,1896
4,131,Une nuit terrible,Short|Comedy|Horror,1896


In [13]:
itm_file.shape

(34513, 4)

## Users dataset

In [14]:
use_file.head()

,userid,twitter_id
0,2,40501255
1,3,417333257
2,4,138805259
3,5,2452094989
4,6,391774225


In [15]:
use_file.shape

(60457, 2)

In [16]:
use_file = use_file.rename(columns = {'userid':'user_id'})

## Ratings dataset

In [17]:
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
816787,60456,8695030,1,1567816456
816788,60456,9398640,1,1569026144
816789,60457,816711,8,1371972851
816790,60458,1559547,2,1373287369
816791,60458,2415464,2,1373772560


In [18]:
rat_file.shape

(816792, 4)

As it shows, the rating timestamp has a special format to compress the information about the date and time when the votation has done.  
Then it is vital to convert this information to a format more comprehensible. 

In [19]:
rat_file['rating_timestamp'] = pd.to_datetime(rat_file['rating_timestamp'], unit='s')
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
816787,60456,8695030,1,2019-09-07 00:34:16
816788,60456,9398640,1,2019-09-21 00:35:44
816789,60457,816711,8,2013-06-23 07:34:11
816790,60458,1559547,2,2013-07-08 12:42:49
816791,60458,2415464,2,2013-07-14 03:29:20


Besides, the column *rating_timestamp* content two fields of information that for our purposes it's more accurate to be used in separated columns, *date* as so *time*.

In [20]:
rat_file = rat_file.rename(columns = {'rating_timestamp':'date'})
rat_file['time'] = rat_file['date'].astype(str)
rat_file['date'] = rat_file['date'].astype(str)

In [21]:
rat_file.time.dtype

dtype('O')

In [22]:
rat_file['date'] = rat_file['date'].str.split(' ').str[0]
rat_file['time'] = rat_file['time'].str[-8:]

In [23]:
rat_file.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


## Connecting the datasets  
Nowadays that all three datasets have a well format so it's time to put together two datasets: **ratings** and **items**.

In [24]:
df = rat_file

In [25]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [26]:
df['date']=df['date'].astype(str)

In [27]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [28]:
df['date']= pd.to_datetime(df['date']) 

In [29]:
df['date'].unique()

array(['2013-07-10T00:00:00.000000000', '2013-07-12T00:00:00.000000000',
       '2013-07-24T00:00:00.000000000', ...,
       '2014-12-11T00:00:00.000000000', '2015-09-25T00:00:00.000000000',
       '2015-09-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [30]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


Just to be fair it could be sort by date the rating dataset. 

In [31]:
df = df.sort_values(by='date')
df.reset_index(drop=True, inplace=True)
df.head()

,user_id,movie_id,rating,date,time
0,48478,938330,10,2013-02-28,17:31:23
1,21390,762125,6,2013-02-28,20:23:38
2,21390,970179,4,2013-02-28,16:40:34
3,31695,1024255,10,2013-02-28,19:43:16
4,30260,2125608,10,2013-02-28,20:56:31


When the dataset rating is available in other temporal variable then we merge both datasets and the link tha unite the information is the id of each movie. It's important to notice that itsn't recommend to use other column like movie title to make linking step because it could be possible to exist movies with the same name.  

In [32]:
df_2 = pd.merge(df,itm_file, on='movie_id')

In [33]:
df_2.head()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year
0,48478,938330,10,2013-02-28,17:31:23,Silent Hill: Revelation 3D,Horror,2012
1,17779,938330,8,2013-03-02,22:09:22,Silent Hill: Revelation 3D,Horror,2012
2,38590,938330,6,2013-03-02,16:30:58,Silent Hill: Revelation 3D,Horror,2012
3,32982,938330,8,2013-03-02,21:02:17,Silent Hill: Revelation 3D,Horror,2012
4,33133,938330,8,2013-03-03,12:37:43,Silent Hill: Revelation 3D,Horror,2012


Finally we add the last dataset, the user information dataset. The linked column is the user id.  
Until now we have constructed a better and robust dataset that contains important information and in the future it's posible to extract more external information using the twitter api.

In [34]:
df_final =pd.merge(df_2, use_file, on='user_id')
df_final = df_final.sort_values(by='date')
df_final.reset_index(drop=True, inplace=True)
df_final.tail()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id
817395,33664,405094,9,2019-11-19,20:03:07,The Lives of Others,Drama|Thriller,2006,1090723140
817396,4054,8772262,7,2019-11-19,06:50:22,Midsommar,Drama|Horror|Mystery|Thriller,2019,130556710
817397,44676,99653,8,2019-11-19,10:00:05,Ghost,Drama|Fantasy|Romance|Thriller,1990,559381134
817398,2998,1950186,8,2019-11-19,00:36:01,Ford v Ferrari,Action|Biography|Drama|Sport,2019,322312595
817399,24378,4729430,10,2019-11-19,22:41:22,Klaus,Animation|Adventure|Comedy|Family,2019,871796913271705600


The only column that looks different in this new dataset is the genre column, so it'll try to change it.  Whit this in mind the idea is to change all this information contained in just one column to multiple categorical columns. Moreover it'll have one column for each genre and according to the genres of each movie this columns could be marqued with 1 if they have the genre and 0 in the oposite case.

In [35]:
df_cross = pd.DataFrame(df_final.genre)

numeration = int(len(df_final))+1
df_cross['id'] = pd.Series(range(1,numeration))

The first step is to create a dataset that only contains the column genre and another column that in this case it called id and this is just an enummeration of each register. It is necessary because in the future transformation it'll be helpful to the purposes.

In [36]:
df_cross.head()

,genre,id
0,Horror,1
1,Action|Thriller,2
2,Action|Thriller,3
3,Crime|Drama|Mystery|Thriller,4
4,Comedy,5


Now here happens the magic. Using the Id of each register it tries to split the values saved in each row of the genre column. On average it could takes between 4-6 minutes to execute this process. (If anyone knows about a more efficient way, please contact to me to fix it)

In [37]:
newdf_cross = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

In [38]:
newdf_cross.columns

Index(['index', 0], dtype='object')

In [39]:
new_dfcross = pd.crosstab(newdf_cross[0], newdf_cross['index'])

In [40]:
new_dfcross.head()

index,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It's posible to look how many and wich are the genres of the films in this dataset as follows.

In [41]:
new_dfcross.columns

Index(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'noGenre'],
      dtype='object', name='index')

Lastly we merge both datasets to have a more robust information.

In [42]:
result = pd.merge(df_final.reset_index(),
                  new_dfcross.reset_index(), 
                  left_index=True, 
                  right_index=True)
result.head()

,index,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,0,48478,938330,10,2013-02-28,17:31:23,Silent Hill: Revelation 3D,Horror,2012,430152300,...,0,0,0,0,0,0,0,0,0,0
1,1,16123,1606378,3,2013-02-28,19:08:48,A Good Day to Die Hard,Action|Thriller,2013,221284547,...,0,0,0,0,0,0,1,0,0,0
2,2,17731,1606378,6,2013-02-28,16:32:04,A Good Day to Die Hard,Action|Thriller,2013,292853430,...,0,0,0,0,0,0,1,0,0,0
3,3,48935,1496422,7,2013-02-28,14:58:23,The Paperboy,Crime|Drama|Mystery|Thriller,2012,36937267,...,0,0,0,0,0,0,1,0,0,0
4,4,28159,302886,9,2013-02-28,23:21:31,Old School,Comedy,2003,200470184,...,0,0,0,0,0,0,0,0,0,0


In [43]:
result.columns

Index([      'index',     'user_id',    'movie_id',      'rating',
              'date',        'time', 'movie_title',       'genre',
        'movie_year',  'twitter_id',             0,      'Action',
             'Adult',   'Adventure',   'Animation',   'Biography',
            'Comedy',       'Crime', 'Documentary',       'Drama',
            'Family',     'Fantasy',   'Film-Noir',   'Game-Show',
           'History',      'Horror',       'Music',     'Musical',
           'Mystery',        'News',  'Reality-TV',     'Romance',
            'Sci-Fi',       'Short',       'Sport',   'Talk-Show',
          'Thriller',         'War',     'Western',     'noGenre'],
      dtype='object')

In [44]:
result = result.drop(['index'], axis=1)
result = result.drop([0], axis=1)
result.head()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,Action,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,48478,938330,10,2013-02-28,17:31:23,Silent Hill: Revelation 3D,Horror,2012,430152300,0,...,0,0,0,0,0,0,0,0,0,0
1,16123,1606378,3,2013-02-28,19:08:48,A Good Day to Die Hard,Action|Thriller,2013,221284547,1,...,0,0,0,0,0,0,1,0,0,0
2,17731,1606378,6,2013-02-28,16:32:04,A Good Day to Die Hard,Action|Thriller,2013,292853430,1,...,0,0,0,0,0,0,1,0,0,0
3,48935,1496422,7,2013-02-28,14:58:23,The Paperboy,Crime|Drama|Mystery|Thriller,2012,36937267,0,...,0,0,0,0,0,0,1,0,0,0
4,28159,302886,9,2013-02-28,23:21:31,Old School,Comedy,2003,200470184,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
itm_file.movie_id.unique().shape

(34511,)

In [46]:
result.movie_id.unique().shape

(34511,)

## Get the count and mean   
The next step is to calculate the total of votes made by each user for the different movies. So, for this purpose we use a temporal variable that refeers the original item dataset. And after that the next step, using the result dataset, we'll get the information counting all the voted films. 

In [47]:
mv_df = itm_file

In [48]:
tes_coid = pd.DataFrame(data = pd.value_counts(result[['movie_id']].values.ravel()),columns=['count'])
tes_coid.shape
tes_coid.reset_index(level=0, inplace=True) 
tes_coid.columns = ['movie_id','count']

In [49]:
tes_coid.head()

,movie_id,count
0,1454468,3067
1,816692,2744
2,993846,2737
3,770828,2663
4,816711,2402


Now it is available the dataset that contains the information of how many votes has for each movie. Just for demostrative pusposes below it shows an espefic title and how it looks, theres are only 3 votes an a mean of 6.33. 

In [50]:
dolar = result['movie_title'] == '$'
result_dolar = result[dolar]
result_dolar

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,Action,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
20131,47433,68152,8,2013-04-07,22:16:20,$,Comedy|Crime|Drama,1971,14782875,0,...,0,0,0,0,0,0,0,0,0,0
230368,4261,68152,7,2014-04-18,12:41:46,$,Comedy|Crime|Drama,1971,78128381,0,...,0,0,0,0,0,0,0,0,0,0
375338,17102,68152,4,2015-04-25,03:05:01,$,Comedy|Crime|Drama,1971,532582706,0,...,0,0,0,0,0,0,0,0,0,0


The the next step is to achieve the rating average of each title. This is posible just grouped the date in id information and then making the mean of all register of this new column.

In [51]:
grouped_data = result.groupby('movie_id')
rating = pd.DataFrame(grouped_data['rating'].mean().round(2))
rating.reset_index(level=0, inplace=True) 
rating.head()

,movie_id,rating
0,10,10.0
1,12,10.0
2,25,8.0
3,91,6.0
4,131,7.0


In [52]:
rating.shape

(34511, 2)

Finally it is posible to merge the three datasets: the test_coid dataset, the rating dataset and the mv_df dataset. At the end we can observe a dataset more or less similar to the ortiginal item dataset but with two new columns: count and rating.

In [53]:
tes_coid = pd.merge(tes_coid,rating, on='movie_id',how='outer')
tes_coid = pd.merge(mv_df,tes_coid, on='movie_id',how='outer')
tes_coid.head()

,movie_id,movie_title,genre,movie_year,count,rating
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.0
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.0
2,25,The Oxford and Cambridge University Boat Race,noGenre,1895,1,8.0
3,91,Le manoir du diable,Short|Horror,1896,3,6.0
4,131,Une nuit terrible,Short|Comedy|Horror,1896,1,7.0


## Construct a simple recommendation system  
The implementation of this model is very trivial.  
Depending to some values it is posible to assing a weight to decide how well or bad a film goes in the opinion of the people. 

* The basic idea behind this system is that movies with a higher probablitiy of being liked by the average audience are more popular in a measurable therms.

### *Weighted Rating (WR)* = ($\frac{v}{v+m}*R$) + ($\frac{m}{v+m}*C$) *  
Where:  
  * v is the number of votes for the movie
  * m is the minimum votes required to be listed in the chart
  * R is the average rating of the movie
  * C is the mean vote across the whole report

In [54]:
vote_counts = tes_coid[tes_coid['count'].notnull()]['count'].astype('int')
vote_avergs = tes_coid[tes_coid['rating'].notnull()]['rating'].astype('int')
C = vote_avergs.mean()
C

6.461014690116768

In [55]:
m= vote_counts.quantile(0.95)
m

90.0

In [56]:
qualified = tes_coid[(tes_coid['count'] >= m) &
                  (tes_coid['count'].notnull())&
                  (tes_coid['rating'].notnull())][['movie_title',
                                                'movie_year',
                                                'count',
                                                'rating',
                                                'genre']]
qualified['count'] = qualified['count'].astype('int')
qualified['rating'] = qualified['rating'].astype('int')
qualified.shape

(1727, 5)

In [57]:
def weighted_rating(x):
    V = x['count']
    R = x['rating']
    return (V/(V+m) * R) +(m/(m+V) * C)

In [58]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [59]:
qualified = qualified.sort_values('wr', ascending = False).head(250)

### Top Movies

In [60]:
qualified.head(15)

,movie_title,movie_year,count,rating,genre,wr
33192,Joker,2019,2236,9,Crime|Drama|Thriller,8.901759
28648,Avengers: Endgame,2019,1363,9,Action|Adventure|Fantasy|Sci-Fi,8.842733
8425,The Shawshank Redemption,1994,998,9,Drama,8.789974
17993,Inception,2010,875,9,Action|Adventure|Sci-Fi|Thriller,8.763203
8271,Forrest Gump,1994,774,9,Drama|Romance,8.735522
2037,12 Angry Men,1957,691,9,Crime|Drama,8.707415
14476,The Dark Knight,2008,664,9,Action|Crime|Drama|Thriller,8.696938
3891,The Godfather,1972,592,9,Crime|Drama,8.664943
9477,The Green Mile,1999,438,9,Crime|Drama|Fantasy|Mystery,8.567218
8118,Schindler's List,1993,415,9,Biography|Drama|History,8.547508


**Note:** This is the principal reason why itsn't correct to use the *movie_title* instead the *movie_id*. Here, we observe that exist two diferents movies with the same title.

In [61]:
joker = itm_file['movie_title'] == 'Joker'
df_joker = itm_file[joker]
df_joker

,movie_id,movie_title,genre,movie_year
21113,1918886,Joker,Comedy|Family|Sci-Fi,2012
33192,7286456,Joker,Crime|Drama|Thriller,2019


In [62]:
itm_file['movie_id'].unique().shape

(34511,)

In [63]:
itm_file['movie_title'].unique().shape

(33119,)

## Construct list of movies depending in the genre  
To do this new aprochment it's necessary to separete the genre column in each value of the movies. Then those are availables.
Finally our function that builds charts for particular genres is posible to declare.

In [64]:
s = tes_coid['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
tes_coid.join(s.apply(lambda x: pd.Series(x.split('|'))))
tes_coid = tes_coid.join(s)
tes_coid.head()

,movie_id,movie_title,genre,movie_year,count,rating,genres
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.0,Documentary
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.0,Short
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.0,Documentary
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.0,Short
2,25,The Oxford and Cambridge University Boat Race,noGenre,1895,1,8.0,noGenre


In [65]:
def build_chart(genre,percentile = 0.85):
    df = tes_coid[tes_coid['genres'] == genre]
    vote_counts = df[df['count'].notnull()]['count'].astype('int')
    vote_averag = df[df['rating'].notnull()]['rating'].astype('int')
    C = vote_averag.mean()
    m = vote_counts.quantile(percentile)
    qualified = df[(df['count']>=m)&
                   (df['count'].notnull()) &
                   (df['rating'].notnull())][['movie_title',
                                              'movie_year',
                                              'count',
                                              'rating',
                                              'genre']]
    qualified['count'] = qualified['count'].astype('int')
    qualified['rating'] = qualified['rating'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x:(x['count']/(x['count']+m)*x['rating'])+(m/(m+x['count'])*C), axis = 1)
    qualified = qualified.sort_values('wr', ascending = False).head(250)
    

    return qualified

### Top 15 Movies according to the genre  
Try to change the genre in the code below.

In [68]:
build_chart('Sci-Fi').head(15)

,movie_title,movie_year,count,rating,genre,wr
28648,Avengers: Endgame,2019,1363,9,Action|Adventure|Fantasy|Sci-Fi,8.853379
17993,Inception,2010,875,9,Action|Adventure|Sci-Fi|Thriller,8.777423
7642,Terminator 2: Judgment Day,1991,234,9,Action|Sci-Fi,8.303429
18410,Gravity,2013,3067,8,Drama|Sci-Fi|Thriller,7.954477
15338,Interstellar,2014,2744,8,Adventure|Drama|Sci-Fi,7.949247
18074,Mad Max: Fury Road,2015,1911,8,Action|Adventure|Sci-Fi|Thriller,7.927873
18151,Star Trek Into Darkness,2013,1879,8,Action|Adventure|Sci-Fi,7.926687
18281,Deadpool,2016,1843,8,Action|Adventure|Comedy|Sci-Fi,7.925305
20490,Her,2013,1771,8,Drama|Romance|Sci-Fi,7.922379
20967,X-Men: Days of Future Past,2014,1752,8,Action|Adventure|Sci-Fi|Thriller,7.921568


## Content based recommendation system

In [ ]:
s = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

#  Pruebas

In [48]:
df_testing = pd.DataFrame(df_final.genre)

pruebas = df_testing.head()
jj = int(pruebas.count()+1)
pruebas['id'] = pd.Series(range(1,jj))

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [49]:
pruebas

,genre,id
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1
1,Drama|Fantasy|Horror|Mystery|Thriller,2
2,Action|Adventure|Fantasy|Horror|Sci-Fi|Thriller,3
3,Action|Adventure|Sci-Fi,4
4,Action|Adventure|Drama|Thriller,5


In [50]:
pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in pruebas.iterrows()]).reset_index()

,index,0
0,Action,1
1,Adventure,1
2,Mystery,1
3,Sci-Fi,1
4,Thriller,1
5,Drama,2
6,Fantasy,2
7,Horror,2
8,Mystery,2
9,Thriller,2


In [51]:
import pandas as pd
df = pd.DataFrame(['a b c']*100000, columns=['col'])
df.head()

,col
0,a b c
1,a b c
2,a b c
3,a b c
4,a b c


In [52]:
print (pd.DataFrame(dict(zip(range(3), [df['col'].apply(lambda x : x.split(' ')[i]) for i in range(3)]))).head())

   0  1  2
0  a  b  c
1  a  b  c
2  a  b  c
3  a  b  c
4  a  b  c


In [53]:
s = pruebas['genre'].str.split('|')
s

0       [Action, Adventure, Mystery, Sci-Fi, Thriller]
1          [Drama, Fantasy, Horror, Mystery, Thriller]
2    [Action, Adventure, Fantasy, Horror, Sci-Fi, T...
3                          [Action, Adventure, Sci-Fi]
4                 [Action, Adventure, Drama, Thriller]
Name: genre, dtype: object

In [54]:
q = pruebas['genre'].str.split('|').tolist()
q

[['Action', 'Adventure', 'Mystery', 'Sci-Fi', 'Thriller'],
 ['Drama', 'Fantasy', 'Horror', 'Mystery', 'Thriller'],
 ['Action', 'Adventure', 'Fantasy', 'Horror', 'Sci-Fi', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Drama', 'Thriller']]

In [55]:
pd.DataFrame(dict(zip(range(16), [pruebas['genre'].apply(lambda x : x.split('|')[i]) for i in range(2)])))

,0,1
0,Action,Adventure
1,Drama,Fantasy
2,Action,Adventure
3,Action,Adventure
4,Action,Adventure


In [56]:
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s

0  0       Action
   1    Adventure
   2      Mystery
   3       Sci-Fi
   4     Thriller
1  0        Drama
   1      Fantasy
   2       Horror
   3      Mystery
   4     Thriller
2  0       Action
   1    Adventure
   2      Fantasy
   3       Horror
   4       Sci-Fi
   5     Thriller
3  0       Action
   1    Adventure
   2       Sci-Fi
4  0       Action
   1    Adventure
   2        Drama
   3     Thriller
dtype: object

In [57]:
s.index = s.index.droplevel(-1)
s.name = 'genres'
s

0       Action
0    Adventure
0      Mystery
0       Sci-Fi
0     Thriller
1        Drama
1      Fantasy
1       Horror
1      Mystery
1     Thriller
2       Action
2    Adventure
2      Fantasy
2       Horror
2       Sci-Fi
2     Thriller
3       Action
3    Adventure
3       Sci-Fi
4       Action
4    Adventure
4        Drama
4     Thriller
Name: genres, dtype: object

In [58]:
# s = pruebas['genre']
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
pruebas.join(s.apply(lambda x: pd.Series(x.split('|'))))
pruebas = pruebas.join(s)
pruebas

,genre,id,genres
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Action
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Adventure
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Mystery
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Sci-Fi
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Thriller
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Drama
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Fantasy
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Horror
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Mystery
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Thriller


In [59]:
uni_gen = pruebas['genres'].unique()
uni_gen

array(['Action', 'Adventure', 'Mystery', 'Sci-Fi', 'Thriller', 'Drama',
       'Fantasy', 'Horror'], dtype=object)

In [60]:
process = pd.get_dummies(pd.Series(list(pruebas['genres'])))

In [61]:
result = pd.merge(pruebas.reset_index(),
                  process.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index_x,genre,id,genres,index_y,Action,Adventure,Drama,Fantasy,Horror,Mystery,Sci-Fi,Thriller
0,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Action,0,1,0,0,0,0,0,0,0
1,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Adventure,1,0,1,0,0,0,0,0,0
2,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Mystery,2,0,0,0,0,0,1,0,0
3,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Sci-Fi,3,0,0,0,0,0,0,1,0
4,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Thriller,4,0,0,0,0,0,0,0,1
5,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Drama,5,0,0,1,0,0,0,0,0
6,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Fantasy,6,0,0,0,1,0,0,0,0
7,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Horror,7,0,0,0,0,1,0,0,0
8,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Mystery,8,0,0,0,0,0,1,0,0
9,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Thriller,9,0,0,0,0,0,0,0,1


In [62]:
new_df = pd.crosstab(result['id'], result['genres'])

In [63]:
new_df

genres,Action,Adventure,Drama,Fantasy,Horror,Mystery,Sci-Fi,Thriller
id,,,,,,,,
1,1,1,0,0,0,1,1,1
2,0,0,1,1,1,1,0,1
3,1,1,0,1,1,0,1,1
4,1,1,0,0,0,0,1,0
5,1,1,1,0,0,0,0,1


In [64]:
data = pd.DataFrame(columns=(uni_gen))
data['id'] = pd.Series(range(1,jj))
data

,Action,Adventure,Mystery,Sci-Fi,Thriller,Drama,Fantasy,Horror,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [65]:
df_final.shape

(816306, 9)

In [66]:
df_final['movie_title'].nunique()
#34314

33093

In [67]:
lista_m = itm_file['movie_title']

lista_f = df_final['movie_title']

In [68]:
df_final[['dID','hID']].groupby(df['mID']).agg(['count', 'size', 'nunique'])

KeyError: "['dID' 'hID'] not in index"

In [ ]:
lista_m.iloc['Joker']

is_male = lista_m.loc[:,] == 'Joker'
df_male = lista_m.loc[is_male]
df_male.head()

In [ ]:
lista_f = pd.DataFrame(lista_f)
lista_f

In [ ]:
#surveys_df.loc[[0, 10], :] surveys_df[surveys_df.year == 2002]

lista_m = pd.DataFrame(lista_m)
lista_m['movie_title'].iloc[2]

In [ ]:
lista_m.movie_title.dtypes

In [ ]:
lista_m.movie_title =='The Oxford and Cambridge University Boat Race'

In [ ]:
is_joker = lista_m.loc[:, 'movie_title'] == 'Upstarts'
df_joker = lista_m.loc[is_joker]
df_joker.head()

In [ ]:
is_joker.unique()

In [ ]:
comparacion = []
for item in lista_m:
    if item in lista_m:
        comparacion.append(item)

if len(comparacion) > 0:
    print('Ambas listas contienen estos elementos')
    for item in comparacion: print('%s' % item)

else:
    print('No existe ningun elemento igual en las listas')

In [ ]:
lista1=["paco","pepe","luis"]

lista2=["diego","mari","luis"]

comparacion = []

 

for item in lista1:

  if item in lista2:

    comparacion.append(item)

 

if len(comparacion) > 0:

  print 'Ambas listas contienen estos elementos'

  for item in comparacion: print '%s' % item

else:

  print 'No existe ningun elemento igual en las listas'

In [ ]:
n_col = df_final.columns

for col in n_col:
    print(df[col].unique)